# Ukraine Tweets Named Entity Recognition (NER)

Named entity recognition (NER) is a subfield of natural language processing (NLP) that involves the identification and extraction of named entities from text. Named entities are real-world objects such as people, organizations, locations, dates, times, quantities, and other types of named entities.

NER algorithms typically use machine learning and statistical models to analyze text and identify spans of text that correspond to named entities. These models can be trained on annotated text data, where human annotators have labeled the named entities in the text.

Once a NER model has identified the named entities in a text, it can be used to perform various tasks, such as information extraction, text classification, and question answering. For example, a NER model could be used to identify the names of companies mentioned in news articles or to extract the dates and times of events from social media posts.

In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [25]:
import os
import csv
import gzip
import gc
import numpy as np
import pandas as pd
import sqlite3
from tqdm import tqdm
import string
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# NLP---
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import translators as ts          
from transliterate import translit, get_available_language_codes
#-------
from multiprocessing import Pool, cpu_count
from IPython.display import clear_output

In [2]:
nlp = spacy.load('en_core_web_sm')
# Download the stopwords from NLTK
# nltk.download('stopwords')

## Load in the desired tweets

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets.db')
# cursor = conn.cursor()

In [ ]:
query = """SELECT * from no_retweets
            WHERE tweetcreatedts BETWEEN '2022-06-01 00:00:00' AND '2022-06-02 00:00:00'"""
df = pd.read_sql(query, conn)
df.head()

In [ ]:
conn.close()

## Data Preprocessing - Data Cleaning

In [30]:
def preprocess_tweet(tweet):
    # Make the tweet lowercase
    tweet = tweet.lower()
    
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    
#     #remove text in square brackets
#     tweet = re.sub(r'\[.*?\]', '', tweet)
    
    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)
    
    # Remove Emojis
    tweet = tweet.encode('ascii', 'ignore').decode('ascii')

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(tweet)
    filtered_tweet = [i for i in tokens if not i in stop_words]
    
    # Remove digits and words with digits
    filtered_tweet = [i for i in filtered_tweet if not any(j.isdigit() for j in i)]
    
    # Joining back the list of words to form a processed tweet
    tweet = ' '.join(filtered_tweet)

    return tweet

In [ ]:
%%time
text_cleaned = df.text.apply(lambda x: preprocess_tweet(x)).to_list()

In [ ]:
df['text'][3]

In [ ]:
text_cleaned[3]

## Named entity recognition (NER) function

In [31]:
def extract_entitites(input: pd.DataFrame) -> pd.DataFrame:
    events_slice=[]
    
    for i, row in input.iterrows():
        doc = nlp(row.text)
        for ent in doc.ents: # This doc property is used for the named entities in the document. 
        #If the entity recognizer has been applied, this property will return a tuple of named entity span objects.
            events_slice += [[ent.text, ent.label_, row.tweetid, 1]]
    return events_slice

## Extract, Preprocess, and Extract Named Entities Workflow - iterative to save memory

In [ ]:
def execute_workflow(dates: str) -> pd.DataFrame():
    # Connect to the SQLite database
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets.db')
    
    query = """SELECT * from no_retweets
            WHERE tweetcreatedts BETWEEN '{}' AND '{}'"""
    query = query.format(dates[0], dates[1])
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    # Process/clean the tweets
    text_cleaned = df.text.apply(lambda x: preprocess_tweet(x)).to_list()
    df['text'] = text_cleaned
    
    # NER function
    ne_data=pd.DataFrame(data=extract_entitites(df),columns=['named_entity','label', 'tweetid', 'count'])
    
    # Load to db
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\ner_labels_id.db')
    
    ne_data.to_sql('ner_labels_id', conn, if_exists='append', index=False)
    
    conn.close()
    
    dates_done = "Last dates done: " + dates[0] + " TO " + dates[1]
    
    return dates_done

In [28]:
# Define start and end dates
start_date = '2022-06-01'
end_date = '2022-08-31'

# Generate date range
dates = pd.date_range(start_date, end_date, freq='D')

# Convert to desired string format
date_strings = dates.strftime('%Y-%m-%d %H:%M:%S').tolist()

date_list = []

# Create a list of lists with sequential dates
date_pairs = [[date_strings[i], date_strings[i + 1]] for i in range(len(date_strings) - 1)]

In [29]:
date_pairs

[['2022-06-01 00:00:00', '2022-06-02 00:00:00'],
 ['2022-06-02 00:00:00', '2022-06-03 00:00:00'],
 ['2022-06-03 00:00:00', '2022-06-04 00:00:00'],
 ['2022-06-04 00:00:00', '2022-06-05 00:00:00'],
 ['2022-06-05 00:00:00', '2022-06-06 00:00:00'],
 ['2022-06-06 00:00:00', '2022-06-07 00:00:00'],
 ['2022-06-07 00:00:00', '2022-06-08 00:00:00'],
 ['2022-06-08 00:00:00', '2022-06-09 00:00:00'],
 ['2022-06-09 00:00:00', '2022-06-10 00:00:00'],
 ['2022-06-10 00:00:00', '2022-06-11 00:00:00'],
 ['2022-06-11 00:00:00', '2022-06-12 00:00:00'],
 ['2022-06-12 00:00:00', '2022-06-13 00:00:00'],
 ['2022-06-13 00:00:00', '2022-06-14 00:00:00'],
 ['2022-06-14 00:00:00', '2022-06-15 00:00:00'],
 ['2022-06-15 00:00:00', '2022-06-16 00:00:00'],
 ['2022-06-16 00:00:00', '2022-06-17 00:00:00'],
 ['2022-06-17 00:00:00', '2022-06-18 00:00:00'],
 ['2022-06-18 00:00:00', '2022-06-19 00:00:00'],
 ['2022-06-19 00:00:00', '2022-06-20 00:00:00'],
 ['2022-06-20 00:00:00', '2022-06-21 00:00:00'],
 ['2022-06-21 00:00:

In [ ]:
%%time
for dates in date_pairs:
    clear_output(wait=True)
    dates_done = execute_workflow(dates)
    
    display(dates_done)

In [5]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\ner_labels_id.db')
# cursor = conn.cursor()

In [6]:
query = """SELECT * from ner_labels_id"""
ne_data = pd.read_sql(query, conn)
conn.close()

In [7]:
ne_data

,named_entity,label,tweetid,count
0,tomorrow,DATE,1531787610854608898,1
1,thursday,DATE,1531787612431826945,1
2,sony,ORG,1531787612431826945,1
3,russian,NORP,1531787613568389125,1
4,russia,GPE,1531787613568389125,1
...,...,...,...,...
5645219,russian,NORP,1564764876555296770,1
5645220,russia,GPE,1564764906859012097,1
5645221,british,NORP,1564764906859012097,1
5645222,lithuania,GPE,1564764908763349003,1


In [ ]:
ne_data_grouped2=ne_data.groupby(['tweetid'],as_index=False).sum().sort_values('count',ascending=False).reset_index(drop=True)
ne_data_grouped2.head(5)

In [ ]:
label_grouped=ne_data.groupby(['label'],as_index=False).sum().sort_values('count',ascending=False).reset_index(drop=True)
label_grouped['pcnt'] = [round(x/label_grouped['count'].sum()*100, 2) for x in label_grouped['count']]

fig = px.bar(label_grouped, x='label', y='pcnt',text_auto=True, labels={"label":"Label", "pcnt":"Percent"})
fig.update_layout(
    title={
        'text': "Extracted Named Entities from Text (percent of total)",
        'x':0.5,
        'font_size':26,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.show()

In [ ]:
# The "pcnt" column is calculated by dividing the count by the sum of counts for each label category. 
# This ensures that the percentages are calculated correctly for each label category.

label_grouped = ne_data[ne_data['label'].isin(['GPE', 'LOC'])].groupby(['named_entity', 'label'], as_index=False).sum()
label_grouped['pcnt'] = round(label_grouped['count'] / label_grouped.groupby('label')['count'].transform('sum') * 100, 2)
label_grouped = label_grouped.sort_values('pcnt', ascending=False)

fig = px.bar(label_grouped.head(25), x='named_entity', y='pcnt', text='pcnt',
             labels={"label": "Label", "pcnt": "Percent"}, color='label') # color_discrete_map={'GPE': 'blue', 'LOC': 'red'}
fig.update_layout(
    title={
        'text': "Extracted named entities from text (percent of total)",
        'x': 0.5,
        'font_size': 26,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.show()

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets.db')
# cursor = conn.cursor()

In [ ]:
query = """SELECT * from no_retweets
            WHERE tweetcreatedts BETWEEN '2022-06-01 00:00:00' AND '2022-08-31 00:00:00'"""
df = pd.read_sql(query, conn)
df

In [ ]:
conn.close()

## Free up some resources

In [ ]:
df = None
del df
gc.collect()

In [ ]:
# for l in labels:
#     df[l] = ""

In [12]:
labels=ne_data['label'].unique().tolist()

for label in labels:
    display(label + " -- " + spacy.explain(label))

'DATE -- Absolute or relative dates or periods'

'ORG -- Companies, agencies, institutions, etc.'

'NORP -- Nationalities or religious or political groups'

'GPE -- Countries, cities, states'

'PERSON -- People, including fictional'

...

'EVENT -- Named hurricanes, battles, wars, sports events, etc.'

'WORK_OF_ART -- Titles of books, songs, etc.'

'QUANTITY -- Measurements, as of weight or distance'

'LAW -- Named documents made into laws.'

'PERCENT -- Percentage, including "%"'

## Enrich this dataframe with more detail from the FULL source

In [8]:
def find_coords_if_present(dates):
        # Connect to the SQLite database
    print('connecting...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\full_summer_tweets.db')
    
    query = """SELECT * from full_summer_tweets
            WHERE tweetcreatedts BETWEEN '{}' AND '{}'"""
    print('selecting...')
    query = query.format(dates[0], dates[1])
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    df = df[~df['coordinates'].isnull()]
    
    dates_done = "Last dates done: " + dates[0] + " TO " + dates[1]
    
    return dates_done, df

In [9]:
def append_coords_if_present(dates, tweets_with_coords):
        # Connect to the SQLite database
    print('connecting...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets.db')
    
    query = """SELECT * from no_retweets
            WHERE tweetcreatedts BETWEEN '{}' AND '{}'"""
    print('selecting...')
    query = query.format(dates[0], dates[1])
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    df['coordinates'] = ""
    for i, row in df.iterrows():
        try:
            df.at[i, 'coordinates'] = str(eval(tweets_with_coords[row.tweetid])['coordinates'])
        except:
            pass
    
    print('writing to new db...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets_coords.db')

    df.to_sql('no_retweets_coords', conn, if_exists='append', index=False)
    conn.close()
    
    dates_done = "Last dates done: " + dates[0] + " TO " + dates[1]
    
    return dates_done

In [10]:
def create_label_dictionary(df):
    # Initialize an empty dictionary
    dictionary = {}

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        key = row['label']
        value = row['named_entity']

        # Check if the key already exists in the dictionary
        if key in dictionary:
            dictionary[key].append(value)
        else:
            dictionary[key] = [value]
    return dictionary

In [160]:
def append_ne_if_present(dates, labels, ne_data):
        # Connect to the SQLite database
    print('connecting...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets_coords.db')
    
    query = """SELECT * from no_retweets_coords
            WHERE tweetcreatedts BETWEEN '{}' AND '{}'"""
    print('selecting...')
    query = query.format(dates[0], dates[1])
    
    df = pd.read_sql(query, conn)
    conn.close()
        # Add column for all possible labels
    for l in labels:
        df[l] = ""
    
    print("Adding labels...")
    for i, row in df.iterrows():
        label_subset_df = ne_data[ne_data['tweetid']==row.tweetid]
        if len(label_subset_df) > 0:
            dictionary = create_label_dictionary(label_subset_df)

            for k,v in dictionary.items():
                    # append list of all named entities by label if present
                df.at[i, k] = str(v) # SQlite doesnt like lists
        else:
            pass
    
    print('writing to new db...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets_coords_labels.db')

    df.to_sql('no_retweets_coords_labels', conn, if_exists='append', index=False)
    conn.close()
    
    dates_done = "Last dates done: " + dates[0] + " TO " + dates[1]
    
    return dates_done

In [ ]:
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\full_summer_tweets.db')
query = """SELECT count(*) from full_summer_tweets"""
pd.read_sql(query, conn)

In [ ]:
%%time
tweets_with_coords = {}

for dates in date_pairs:
    display(dates_done)
    
    dates_done, df= find_coords_if_present(dates)
    if len(df) > 0:
        for i,row in df.iterrows():
            tweets_with_coords[row.tweetid] = row.coordinates
    else:
        pass
    
    clear_output(wait=True)   

In [ ]:
print("Percent of tweets (16,203) with coordinates in full summer tweets:")
round(len(tweets_with_coords)/14829283 * 100, 2) 

In [ ]:
%%time

for dates in date_pairs:
    display(dates_done)
    
    dates_done = append_coords_if_present(dates, tweets_with_coords)
    
    clear_output(wait=True) 

In [134]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets_coords_labels.db')
query = """SELECT * from no_retweets_coords_labels LIMIT 10"""
df = pd.read_sql(query, conn)
conn.close()
df

,location,tweetid,tweetcreatedts,retweetcount,text,hashtags,coordinates,DATE,ORG,NORP,...,PRODUCT,LOC,LANGUAGE,MONEY,FAC,EVENT,WORK_OF_ART,QUANTITY,LAW,PERCENT
0,The Poletti Group LLC is a company out of Star...,1531787610854608898,2022-06-01 00:00:00,0,#ArchaicAgeofDarkess #tabletopgames #ttrpgcomm...,"[{'text': 'ArchaicAgeofDarkess', 'indices': [0...",,['tomorrow'],,,...,,,,,,,,,,
1,"Florence, SC",1531787611337068544,2022-06-01 00:00:00,0,#dailymotivation It doesn't matter how much y...,"[{'text': 'dailymotivation', 'indices': [0, 16...",,,,,...,,,,,,,,,,
2,"Beijing,China",1531787611978625027,2022-06-01 00:00:00,0,Still remember our #innovative #business model...,"[{'text': 'innovative', 'indices': [19, 30]}, ...",,,,,...,,,,,,,,,,
3,None,1531787611978846209,2022-06-01 00:00:00,0,⚽️Algovenger World Cup Soccer⚽️\n\nLet’s Go!!!...,"[{'text': 'USA', 'indices': [71, 75]}, {'text'...",,,,,...,,,,,,,,,,
4,Worldwide,1531787612431826945,2022-06-01 00:00:00,2,Assassins Creed Origins 60FPS Update Coming Th...,"[{'text': 'AssassinsCreedOrigins', 'indices': ...",,['thursday'],['sony'],,...,,,,,,,,,,
5,None,1531787613568389125,2022-06-01 00:00:00,0,WATCH: #Russian Ka-52 Attack Helicopter’s Ludi...,"[{'text': 'Russian', 'indices': [7, 15]}, {'te...",,,,['russian'],...,,,,,,,,,,
6,Texas,1531787613711081472,2022-06-01 00:00:01,1,Scientists boost the levels of THC in cannabis...,"[{'text': 'Medical', 'indices': [63, 71]}, {'t...",,['today'],,,...,,,,,,,,,,
7,"Philadelphia, PA",1531787614088531968,2022-06-01 00:00:01,0,Read: https://t.co/6CytIXStUX\n.\n.\n.\n.\n.\n...,"[{'text': 'HighSchoolReunion', 'indices': [40,...",,,,,...,,,,,,,,,,
8,"Quito, Ecuador",1531787615082528770,2022-06-01 00:00:01,6,#Russia Did Everything To Avert Global Food Cr...,"[{'text': 'Russia', 'indices': [0, 7]}]",,,,"['russian', 'russian']",...,,,,,,,,,,
9,Bitcoin,1531787617091600384,2022-06-01 00:00:01,0,"May 31, 2022 23:59:00 UTC | 31,772.00$ | 29,56...","[{'text': 'Bitcoin', 'indices': [67, 75]}, {'t...",,,['utc'],,...,,,,,,,,,,


In [ ]:
print("Percent of tweets (2,631) with coordinates in summer original tweets:")
round(len(df[df['coordinates'] != ""])/3191820  * 100, 2) 

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets.db')
query = """SELECT count(*) from no_retweets"""
pd.read_sql(query, conn)

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\ner_labels_id.db')
# cursor = conn.cursor()

In [ ]:
query = """SELECT * from ner_labels_id"""
ne_data = pd.read_sql(query, conn)
conn.close()

### Attach labels

In [14]:
%%time
dates_done = ''
for dates in date_pairs:
    display(dates_done)
    
    dates_done = append_ne_if_present(dates, labels, ne_data)
    
    clear_output(wait=True) 

Wall time: 3h 19min 19s


### Determine if location field is a legitimate location/POI (GPE, LOC, FAC)

In [167]:
def ner_from_location_field(df):
    locs=[]
    labels=[]

    df['places'] = ''
    df['p_labels'] = ''
    df = df[['location', 'places', 'p_labels', 'tweetid', 'tweetcreatedts', 'retweetcount', 'text', 'hashtags', 'coordinates', 'DATE', 'ORG', 'NORP', 'GPE', 'PERSON', 'TIME', 'CARDINAL', 'ORDINAL', 'PRODUCT', 'LOC', 'LANGUAGE', 'MONEY', 'FAC', 'EVENT', 'WORK_OF_ART', 'QUANTITY', 'LAW', 'PERCENT']]

    for i, row in df.iterrows():
        loc = row.location
        try:
            if ',' in loc:
                loc = ' '.join(loc.split(','))

            clean = preprocess_tweet(loc)
            doc = nlp(clean)

            if len(doc.ents)==0:
                locs.append('')
                labels.append('')

            else:
                et = []
                el = []
                for ent in doc.ents: # This doc property is used for the named entities in the document. 
                        #If relevant to 'place'
                    if ent.label_ in ['GPE', 'LOC', 'FAC']:
                        et.append(ent.text)
                        el.append(ent.label_)
                    else:
                        pass

                locs.append(et)
                labels.append(el)
        except: # if location is None
            locs.append('')
            labels.append('')

    df['places'] = locs
    df['p_labels'] = labels
    
    return df

In [168]:
def append_ne_from_loc_if_present(dates):
        # Connect to the SQLite database
    print('connecting...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\no_retweets_coords_labels.db')
    
    query = """SELECT * from no_retweets_coords_labels
            WHERE tweetcreatedts BETWEEN '{}' AND '{}'"""
    print('selecting...')
    query = query.format(dates[0], dates[1])
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    df = ner_from_location_field(df)

    print('writing to new db...')
    conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\summer_2022_no_retweets_final.db')
    
    df['places'] = df['places'].astype(str)
    df['p_labels'] = df['p_labels'].astype(str)
    
    df.to_sql('summer_2022_no_retweets_final', conn, if_exists='append', index=False)
    conn.close()
    
    dates_done = "Last dates done: " + dates[0] + " TO " + dates[1]
    
    return dates_done

In [169]:
%%time
dates_done = 'first'
for dates in date_pairs:
    display(dates_done)
    
    dates_done = append_ne_from_loc_if_present(dates)
    
    clear_output(wait=True) 

Wall time: 1h 57min 6s


## Subset all FULL tweets to a June-Aug SQlite table

In [ ]:
%%time
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\full_tweets.db')

print('selecting from full...')
query = """SELECT * from full_tweets
            WHERE tweetcreatedts BETWEEN '2022-06-01 00:00:00' AND '2022-08-31 00:00:00'"""
df = pd.read_sql(query, conn)
conn.close()
print("done selecting from full...")

print('writing to new db...')
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\full_summer_tweets.db')

df.to_sql('full_summer_tweets', conn, if_exists='append', index=False)
conn.close()

df = None
del df
gc.collect()